## Precarga de librerias y funciones

In [1]:
%run "../recurrentes.ipynb"

In [2]:
%run "../funciones.ipynb"

In [3]:
# # Preseteo optuna especificar bucket y nombre del estudio


# nombre_archivo = 'optimization_tree.db'
# bucket = 'b2/'


# estudio_optuna = base_path + 'buckets/' + bucket + 'optimization_tree.db'


# # cargar estudio
# # a) competencia_02
# # b) competencia_02_lags
# # c) competencia_02_lags_y_deltas

# nombre_estudio = 'competencia_02_lags'


In [4]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l

# df_train = pd.read_parquet(dataset_clase_ternaria_l)
df_train = pd.read_parquet(dataset_lags_clase_ternaria_l)
# df_lag_delta = pd.read_parquet(dataset_lags_deltas_y_clase_ternaria_l)


# Light GBM

## Formateo pre modelo

In [5]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [6]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202108


In [7]:
data = df_train

In [8]:
df_train.shape

(4735593, 378)

In [9]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [10]:
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [11]:
# Especificar mes de train y test

df_train = data[data['foto_mes']<=202104]
df_test = data[data['foto_mes']==202106]


In [12]:
print(data.shape)  # Número de filas y columnas
print(data.info(memory_usage='deep'))

(4735593, 380)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4735593 entries, 0 to 4735592
Columns: 380 entries, numero_de_cliente to clase_binaria
dtypes: bool(2), float64(313), int32(1), int64(63), object(1)
memory usage: 13.6 GB
None


In [13]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [14]:
w_train = df_train.loc[X_train.index, 'clase_peso']


### Back testing 

In [15]:
# Ordenar entradas en df_train en base a la columna foto_mes de meses más antiguos a más recientes
X_train = X_train.sort_values('foto_mes', ascending=True)

In [16]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)



params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[0],
)

cv_results





MemoryError: Unable to allocate 11.4 GiB for an array with shape (4075500, 377) and data type float64

In [ ]:
max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final =  max_gan * 5
final

476000.0

In [ ]:
# RESULTADOS:
# 1. Modelo Base = 10737406704.0

### Optimización

In [ ]:
# 4 - Optimizacion

def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 8, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.00005, 0.3)  # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train,
                             label=Y_train,  # elegir la clase
                             weight=w_train)
                             
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=300,  # modificar, subir y subir... y descomentar la línea inferior
        
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

storage_name = rf"sqlite:///{estudio_optuna}"

study_name = nombre_estudio



study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

import optuna

class EarlyStoppingByImprovement:
    def __init__(self, patience: int):
        """
        Args:
            patience (int): Número de pruebas consecutivas sin mejora antes de detener la optimización.
        """
        self.patience = patience
        self.best_value = None
        self.no_improvement_trials = 0

    def __call__(self, study, trial):
        current_best_value = study.best_trial.value

        # Si la métrica mejora, reseteamos el contador
        if self.best_value is None or current_best_value > self.best_value:
            self.best_value = current_best_value
            self.no_improvement_trials = 0
        else:
            # Si no hay mejora, incrementamos el contador
            self.no_improvement_trials += 1

        # Detener si no ha habido mejora en 'self.patience' pruebas consecutivas
        if self.no_improvement_trials >= self.patience:
            print(f"Early stopping: No hay mejora en {self.patience} pruebas consecutivas.")
            study.stop()

# Crear el callback con 100 pruebas consecutivas sin mejora
early_stopping_callback = EarlyStoppingByImprovement(patience=100)

# Crear el estudio
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

# Optimizar el estudio con el callback personalizado
study.optimize(objective, n_trials=30000)

[I 2024-11-08 11:24:00,504] Using an existing study with name 'competencia_02' instead of creating a new one.
[I 2024-11-08 11:24:00,548] Using an existing study with name 'competencia_02' instead of creating a new one.
[I 2024-11-08 11:25:26,175] Trial 1 finished with value: 484022000.0 and parameters: {'num_leaves': 73, 'learning_rate': 0.048381168713106194, 'min_data_in_leaf': 1888, 'feature_fraction': 0.9611290073992866, 'bagging_fraction': 0.5702627562564301}. Best is trial 1 with value: 484022000.0.
[I 2024-11-08 11:26:34,198] Trial 2 finished with value: 454993000.0 and parameters: {'num_leaves': 28, 'learning_rate': 0.03703877249854557, 'min_data_in_leaf': 393, 'feature_fraction': 0.2450786254076267, 'bagging_fraction': 0.23471371488088055}. Best is trial 1 with value: 484022000.0.
[I 2024-11-08 11:28:04,561] Trial 3 finished with value: 489223000.0 and parameters: {'num_leaves': 79, 'learning_rate': 0.05486123634370735, 'min_data_in_leaf': 1380, 'feature_fraction': 0.618491354

KeyboardInterrupt: 

## Voting GD

In [ ]:
# 6 - Voting promedio de modelos para reducir varianza

best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")

train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

df_voting = pd.DataFrame()

for semilla in numeros_random:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_params['num_leaves'],
    'learning_rate': study.best_params['learning_rate'],
    'min_data_in_leaf': study.best_params['min_data_in_leaf'],
    'feature_fraction': study.best_params['feature_fraction'],
    'bagging_fraction': study.best_params['bagging_fraction'],
    'seed': semilla,
    'verbose': 0
}
    
    
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    df_voting[f'semilla_{semilla}'] = model.predict(X_test)

df_voting['promedio'] = df_voting.mean(axis=1)
    

Mejor cantidad de árboles para el mejor model 300


C:\Users\Admin\AppData\Local\Temp\ipykernel_2908\1798680220.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_voting['promedio'] = df_voting.mean(axis=1)


## Predicción

In [ ]:
# 7 - Predicción del modelo 

y_pred_lgm = df_voting['promedio']
y_pred_labels = (y_pred_lgm > 0.025).astype(int)

In [ ]:
# 8 - Dataframe con las variables más importantes
importancias = model.feature_importance()
nombres = model.feature_name()
df_resultado = pd.DataFrame({
    'numero_de_cliente': X_test['numero_de_cliente'],
    'Predicted': y_pred_labels
})

## Entrega Kaggle

In [ ]:
#Ingresar el path sobre el cual se quiere ingresar el archivo a entregar
entrega = 'entrega_kaggle'
formato = '.csv'
numero = entrega+'_001'+formato
entrega_final = df_resultado.to_csv(rf"{save_path}{numero}}", index=False)
best_params = study.best_params

In [ ]:
cantidad_columnas = df_gbm.shape[1]
message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}"

<!-- ### Kaggle -->

In [ ]:

competition= 'DMEyF-2024-Segunda'

In [ ]:
kaggle.api.competition_submit(competition=competition, file_name= numero, message=message, quiet=False)

### Puntos de corte

In [ ]:

# predicciones = y_pred_lgm

# X_test['Probabilidad'] = predicciones

# tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

# cortes = range(9000,14000,100)

# num_subida_kaggle = 65
# for envios in cortes:
    
#     tb_entrega['Predicted'] = 0
#     tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
#     resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
#     print("Cantidad de clientes {}".format(envios))
    
#     nombre_archivo = 'entrega_0{}.csv'.format(num_subida_kaggle)
#     entrega_final = os.path.join(path, nombre_archivo)
#     resultados.to_csv(entrega_final, index=False)
    
    
#     cantidad_columnas = df_train.shape[1]
#     message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}, punto de corte: {envios}, optimizado con optuna: {study_name}"
    
#     num_subida_kaggle += 1
    
    
    
#     entrega_final = os.path.join(path, nombre_archivo)   
#     competencia = 'dm-ey-f-2024-primera'
#     try:
#         api.competition_submit(file_name=entrega_final,message=message,competition=competencia)
#     except:
#         print(f"Numero máximo de envios, último envio ={num_subida_kaggle}")

In [ ]:
# df_train.shape

In [ ]:
# nombre_modelo = 'lgbm_e_en_abril_p_en_junio_451_features.txt'
# model.save_model(rf"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\modelos_lgbm\{nombre_modelo}")